In [ ]:
# For data manipulation and analysis
import pandas as pd
import numpy as np

# For model evaluation
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

In [ ]:
# clusters, tags, sentiment
tags_c = pd.read_csv('../dataset/final_clusters.csv')
tags = pd.read_csv("../dataset/tags_withglovevec.csv")
df_s = pd.read_csv('../dataset/df_tag_sentiment.csv')


# By Fold Evaluation Results

## Train-Test

In [ ]:
# calculation of similarity
def cosine_similarity(v1, v2):
    '''Cosine similarity function only computes similarity for NON-NAN values'''
    # Indices where both v1 and v2 are not NaN
    shared_idx = np.where(~np.isnan(v1) & ~np.isnan(v2))

    # If no shared indices, return 0
    if len(shared_idx[0]) == 0:
        return 0

    # Extract shared values
    v1_shared = v1[shared_idx]
    v2_shared = v2[shared_idx]

    # Compute the dot product and norms only on shared values
    dot_product = np.dot(v1_shared, v2_shared)
    norm_v1 = np.linalg.norm(v1_shared)
    norm_v2 = np.linalg.norm(v2_shared)

    # Prevent division by zero
    if norm_v1 == 0 or norm_v2 == 0:
        return 0

    return dot_product / (norm_v1 * norm_v2)


def get_sentiment_label(sentiment):
    if sentiment > 0.5:
        return "positive"
    elif sentiment < -0.5:
        return "negative"
    else:
        return "neutral"


def get_dominant_sentiment(user_sentiments):
    pos_count = sum(1 for s in user_sentiments if s > 0.5)
    neg_count = sum(1 for s in user_sentiments if s < -0.5)
    neutral_count = len(user_sentiments) - pos_count - neg_count

    if pos_count > neg_count and pos_count > neutral_count:
        return "positive"
    elif neg_count > pos_count and neg_count > neutral_count:
        return "negative"
    else:
        return "neutral"


def get_k_nearest_neighbors(target_user, matrix, k):
    similarities = {}

    for user in matrix.index:
        if user == target_user:
            continue
        common_movies = matrix.loc[target_user].dropna().index.intersection(matrix.loc[user].dropna().index)
        if len(common_movies) > 0:
            user_vector = matrix.loc[target_user][common_movies].values.reshape(1, -1)
            target_vector = matrix.loc[user][common_movies].values.reshape(1, -1)
            sim = cosine_similarity(user_vector, target_vector)

            similarities[user] = sim

    # Sort by similarity
    sorted_neighbors = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    return sorted_neighbors[:k]


def get_distributed_recommendations(target_user, matrix, new_to_original_index, df_sent, k):
    # Get the sentiments for movies rated by the target user
    user_sentiments = matrix.loc[target_user].dropna()

    # Determine the dominant sentiment of the target user
    dominant_sentiment = get_dominant_sentiment(user_sentiments)

    # Get the k-most similar users
    similar_users = get_k_nearest_neighbors(target_user, matrix, k)
    sentiment_buckets = defaultdict(list)
    # For each similar user, gather movies they've rated

    for user, similarity in similar_users:
        # Map the user index to its original userId
        original_user_id = new_to_original_index[user]

        for movie, sentiment in matrix.loc[user].items():
            if not np.isnan(sentiment):

                # Filter df_sent to get rows for the current user and movie
                user_movie_df = df_sent[(df_sent['userId'] == original_user_id) & (df_sent['movieId'] == movie)]

                # If there are no sentiments for this user-movie pair, continue
                if user_movie_df.empty:
                    continue

                # Aggregate the scaled_sentiment_values. Here, I'm using mean, but you can adjust this
                movie_sentiment = user_movie_df['scaled_sentiment_value'].mean()

                sentiment_direction = get_dominant_sentiment([movie_sentiment])
                sentiment_buckets[sentiment_direction].append((movie, movie_sentiment))

    return sentiment_buckets[dominant_sentiment][:k]  # returns movieId and sentiment value - scaled sentiment value from df_sent


def generate_and_store_recommendations(df_rec, mat, new_to_original_index, df_sent, k=10):
    for idx, row in df_rec.iterrows():
        original_user_id = row['userId']

        # Map the original userId to the new continuous index
        target_user = original_to_new_index.get(original_user_id, None)

        # If the user exists in the matrix
        if target_user is not None:
            # Get recommendations
            recommendations = get_distributed_recommendations(target_user, mat, new_to_original_index, df_sent, k)

            # Store recommendations in df_rec under the original userId
            df_rec.at[idx, 'recommendations'] = recommendations


# Define sentiment category function
def determine_sentiment_category(sentiment_value):
    if -1 <= sentiment_value < -0.5:
        return 'Negative'
    elif -0.5 <= sentiment_value <= 0.5:
        return 'Neutral'
    else:
        return 'Positive'


# Extract tags function
def get_tags_for_movie(movieId, sentiment_value):
    sentiment_category = determine_sentiment_category(sentiment_value)

    # Filter rows with matching movieId and sentiment category
    tags = df_comb_t[(df_comb_t['movieId'] == movieId) &
                     (df_comb_t['scaled_sentiment_value_avg'].apply(
                         determine_sentiment_category) == sentiment_category)]['tag'].tolist()

    # Convert tags list to set and back to list to ensure distinct tags
    distinct_tags = list(set(tags))

    return (movieId, distinct_tags)

def get_user_data(df_rec, tag_cv, target_userId):
    """
    Retrieve movie recommendations, user tags, and tags_movies for a target userId.

    Parameters:
        df_rec (DataFrame): The DataFrame containing userId, recommendations, and tags_movies columns.
        tag_cv (DataFrame): The DataFrame containing userId, movieId, and tags.
        target_userId (int): The userId for whom the data is to be fetched.

    Returns:
        tuple: A tuple containing three lists:
               - The first list contains recommendations (movieId, sentiment) for the target userId.
               - The second list contains tags applied by the target userId in the format (movieId, tag).
               - The third list contains tags_movies entries for the target userId in the format [(movieId, [tags...]), ...].
    """
    # Fetch movie recommendations
    recommendations = get_user_recommendations(df_rec, target_userId)

    # Fetch tags applied by the user
    user_tags = get_user_tags(tag_cv, target_userId)

    # Fetch tags_movies data for the user
    tags_movies = get_tags_movies_for_user(df_rec, target_userId)

    return recommendations, user_tags, tags_movies


def get_user_recommendations(df_rec, target_userId):
    filtered_df = df_rec[df_rec['userId'] == target_userId]
    if len(filtered_df) == 0:
        return []
    recommendations = filtered_df['recommendations'].iloc[0]
    return recommendations


def get_user_tags(tag_cv, target_userId):
    filtered_df = tag_cv[tag_cv['userId'] == target_userId]
    if len(filtered_df) == 0:
        return []
    user_tags = filtered_df[['movieId', 'tag']].values.tolist()
    return user_tags


def get_tags_movies_for_user(df_rec, target_userId):
    filtered_df = df_rec[df_rec['userId'] == target_userId]
    if len(filtered_df) == 0:
        return []
    tags_movies = filtered_df['tags_movies'].iloc[0]
    return tags_movies


def get_user_feature_set_and_wsd(df_feat, df_wsd, user_tags, tag_clusters, target_userId):
    """
    Fetch additional features, WSD, and TF/TF-IDF info for a list of user tags.
    
    Parameters:
        df_feat (DataFrame): The DataFrame containing features for each tag.
        df_wsd (DataFrame): The DataFrame containing WSD information.
        user_tags (list): The list containing tags applied by the user in the format (movieId, tag).
        target_userId (int): The userId for whom the WSD data is to be fetched.
        
    Returns:
        tuple: A tuple containing three DataFrames:
               - The first DataFrame contains additional features for each user tag.
               - The second DataFrame contains WSD information for the user.
               - The third DataFrame contains TF and TF-IDF information for the user tags.
    """
    # Filtering df_feat to only include rows where the tag is in user_tags
    filtered_df_feat = df_feat[df_feat['tag'].isin([tag for _, tag in user_tags])]

    # Filtering df_wsd to include only rows for the target_userId
    filtered_df_wsd = df_wsd[df_wsd['userId'] == target_userId]

    # joining the cluster to the tags

    # Create a set of (movieId, tag) tuples for easier filtering
    user_tags_set = set(tuple(x) for x in user_tags)

    return filtered_df_feat, filtered_df_wsd




def calculate_similarity_for_recommendations(recommendations, tags_movies, df_feat, user_features, df_wsd, user_wsd):
    similarity_list = []

    # Create a dictionary to easily fetch movie tags
    tags_movies_dict = dict(tags_movies)

    for movieId, _ in recommendations:
        movie_tags = tags_movies_dict.get(movieId, [])  # Fetch the tags from the tags_movies dictionary
        if not movie_tags:
            continue

        for user_tag in user_features['tag'].unique():
            for movie_tag in movie_tags:

                user_wsd_row = user_wsd[user_wsd['tag'] == user_tag]
                movie_wsd_row = df_wsd[(df_wsd['movieId'] == movieId) & (df_wsd['tag'] == movie_tag)]

                # nested loop to calculate the wsd
                wsd_similarities = []
                for user_sense in user_wsd_row['disambiguated_sense'].values:
                    for movie_sense in movie_wsd_row['disambiguated_sense'].values:
                        wsd_similarities.append(user_sense == movie_sense)

                if wsd_similarities:
                    wsd_similarity = np.mean(wsd_similarities)
                else:
                    wsd_similarity = 0

                wsd_confidence = movie_wsd_row['confidence'].mean() if not movie_wsd_row.empty else 0

                user_feat_row = user_features[user_features['tag'] == user_tag]
                movie_feat_row = df_feat[df_feat['tag'] == movie_tag]

                pos_match = (user_feat_row['POS'].values == movie_feat_row['POS'].values).mean() if user_feat_row['POS'].values.size and movie_feat_row['POS'].values.size else 0
                ner_match = (user_feat_row['ner_label'].values == movie_feat_row['ner_label'].values).mean() if user_feat_row['ner_label'].values.size and movie_feat_row['ner_label'].values.size else 0
                sentiment_match = (user_feat_row['sentiment_label'].values == movie_feat_row['sentiment_label'].values).mean() if user_feat_row['sentiment_label'].values.size and movie_feat_row['sentiment_label'].values.size else 0

                # Check for cluster match
                cluster_match = int(user_feat_row['cluster'].values[0] == movie_feat_row['cluster'].values[0]) if user_feat_row['cluster'].values.size and movie_feat_row['cluster'].values.size else 0

                similarity_list.append({
                    'movieId': movieId,
                    'user_tag': user_tag,
                    'movie_tag': movie_tag,
                    'wsd_similarity': wsd_similarity,
                    'wsd_confidence': wsd_confidence,
                    'pos_match': pos_match,
                    'ner_match': ner_match,
                    'sentiment_match': sentiment_match,
                    'cluster_match': cluster_match  # Adding the cluster match to the dataframe
                })

    similarity_df = pd.DataFrame(similarity_list)
    return similarity_df


def calculate_total_similarity(row, weights):
    return sum(row[metric] * weight for metric, weight in weights.items())

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# reading in fold 1-5
for fold in range(1,5):
    df_fold_train_1 = pd.read_csv(f"../dataset/evaluation_folds/fold_{fold}_train.csv")
    tags_c = pd.read_csv('../dataset/final_clusters.csv')
    tags = pd.read_csv("../dataset/tags_withglovevec.csv")
    df_s = pd.read_csv('../dataset/df_tag_sentiment.csv')
    
    # data conversions
    tags_c['tag'] = tags_c['tag'].astype('str')
    df_s['tag'] = df_s['tag'].astype('str')
        
    # combine df_comb and tags_c
    df_comb = tags_c.merge(df_s, on='tag', how='left')
    
    # drop duplicates on (tag, movie) -> so the cluster average is not bias to duplicate tags per movie
    df_comb.drop_duplicates(subset='tag', inplace=True)
    # df_comb
    avg_sentiment_by_cluster = df_comb.groupby('cluster')['scaled_sentiment_value'].mean().reset_index()
    
    df_comb = pd.merge(df_comb, avg_sentiment_by_cluster, on='cluster', suffixes=('', '_cluster_avg'))
    df_comb = df_comb.drop(columns=['glove_vec'])
    df_comb['scaled_sentiment_value_avg'] = df_comb['scaled_sentiment_value_cluster_avg']
    df_mat = tags.merge(df_comb, on=['tag'], how='inner')
    
    df_sent = df_mat  # take copy for later on 
  
    #### CF generate recommendations and store in a file here - 
    # df_fold_train_1
    # tags_c
    tags_c['cluster'] = tags_c['new_cluster']
    tags_c.drop(columns='new_cluster', inplace=True)
    
    df_s = pd.read_csv('../dataset/df_tag_sentiment.csv')
    
    # data conversions
    tags_c['tag'] = tags_c['tag'].astype('str')
    df_s['tag'] = df_s['tag'].astype('str')
        
    # combine df_comb and tags_c
    df_comb = tags_c.merge(df_s, on='tag', how='left')
    
    # drop duplicates on (tag, movie) -> so the cluster average is not bias to duplicate tags per movie
    df_comb.drop_duplicates(subset='tag', inplace=True)
    
    avg_sentiment_by_cluster = df_comb.groupby('cluster')['scaled_sentiment_value'].mean().reset_index()
    
    df_comb = pd.merge(df_comb, avg_sentiment_by_cluster, on='cluster', suffixes=('', '_cluster_avg'))
    
    df_mat = df_fold_train_1.merge(df_comb, on=['tag'], how='inner')
    df_sent = df_mat
    # df_mat
    df_mat['scaled_sentiment_value_avg'] = df_mat['scaled_sentiment_value_cluster_avg']
    
    # Convert the 'scaled_sentiment_value_avg' column to float
    df_mat['scaled_sentiment_value_avg'] = df_mat['scaled_sentiment_value_avg'].astype('float')
    
    # Keep only relevant columns
    df_mat = df_mat[["userId", "movieId", "scaled_sentiment_value_avg"]]
    
    # Check for duplicates and print them
    duplicates = df_mat[df_mat.duplicated(subset=['userId', 'movieId'], keep=False)]
    
    # Group by 'userId' and 'movieId' to get the average 'scaled_sentiment_value_avg'
    df_mat = df_mat.groupby(['userId', 'movieId'])['scaled_sentiment_value_avg'].mean().reset_index()
    
    # Create the pivot table
    mat = pd.pivot_table(df_mat, values='scaled_sentiment_value_avg', index=['userId'], columns=['movieId'])
    
    # Assuming 'mat' is your matrix
    # Resetting the index will add a column 'userId' with the original values
    mat = mat.reset_index()
    
    # creating a dictionary with original userId and new continuous index
    original_to_new_index = {old_id: new_id for new_id, old_id in enumerate(mat['userId'])}
    
    # assign new continous index to the userId column 
    mat['userId'] = mat['userId'].map(original_to_new_index)
    
    # Now, set 'userId' as the index again
    mat.set_index('userId', inplace=True)
    
    user_indices = mat.index[mat.apply(lambda x: (x.count() == 1) and (x.dropna().iloc[0] > 0.5), axis=1)]
    
    new_to_original_index = {v: k for k, v in
                             original_to_new_index.items()}  # this exists outside the function to ensure the new_to_original_index exists for mapping
    
    original_to_new_index = {v: k for k, v in new_to_original_index.items()}
    df_comb['scaled_sentiment_value_avg'] = df_comb['scaled_sentiment_value_cluster_avg']
    # creation of df_rec dataframe 
    
    df_rec = df_sent[["userId"]].drop_duplicates().reset_index(drop=True)
    
    # Initialize recommendations column in df_rec
    df_rec['recommendations'] = None
    
    # Generate and store recommendations
    df_rec['recommendations'] = generate_and_store_recommendations(df_rec, mat, new_to_original_index, df_sent)
    df_rec['recommendations'] = df_rec['recommendations'].apply(lambda x: [(int(a), round(b, 4)) for a, b in x])
    
    df_comb_t = pd.merge(tags, df_comb, on='tag', how='left')
    df_comb_t = df_comb_t[["movieId", "tag", "scaled_sentiment_value", "scaled_sentiment_value_avg"]]
    
    # getting initial tags recommendation 
    # Apply function on recommendations column
    df_rec['tags_movies'] = df_rec['recommendations'].apply(
        lambda recs: [get_tags_for_movie(movieId, sentiment_value) for movieId, sentiment_value in recs])
    df_rec.to_json(f"../dataset/evaluation/cf_train_{fold}_recs.json")
    
    
    df_feat = pd.read_json("../dataset/df_feat.json")
    tag_cv = pd.read_json('../dataset/tag_csv.json')
    tag_clusters = pd.read_csv("../dataset/tag_clusters.csv")
    
    df_wsd = pd.read_csv("../dataset/df_wsd.csv")
    # df_wsd  # can directly filter on this
    # df_feat  # need to merge on tag_clusters
    df_feat_cl = pd.merge(df_feat, tag_clusters, on=['tag'], how='left')

            
    precision_scores = []
    recall_scores = []
    accuracy_scores = []
    
    # df_feat
    
    test_data = pd.read_csv(f'../dataset/evaluation_folds/fold_{fold}_test.csv')
    train_data = df_fold_train_1
    
    df_fold_1_results = pd.DataFrame(columns=['userId', 'recommended_tags'])
    
    ############ CB MODEL ############################
    
    merged_tag_cv = pd.merge(tag_cv, test_data, on=['userId', 'movieId', 'tag'], how='outer', indicator=True)
    
    # Now, filter out the rows that come only from the 'test_data' using the indicator column.
    tag_cv_filtered = merged_tag_cv[merged_tag_cv['_merge'] != 'right_only'].drop(columns=['_merge'])
    
    # Repeat the process for df_wsd.
    merged_df_wsd = pd.merge(df_wsd, test_data, on=['userId', 'movieId', 'tag'], how='outer', indicator=True)
    df_wsd_filtered = merged_df_wsd[merged_df_wsd['_merge'] != 'right_only'].drop(columns=['_merge'])
    
    for userId in train_data['userId'].unique():
        try:
            target_user = userId
    
            recommendations, user_tags, tags_movies = get_user_data(df_rec, tag_cv_filtered, target_user)
    
            user_features, user_wsd = get_user_feature_set_and_wsd(df_feat, df_wsd_filtered, user_tags, tag_clusters,
                                                                   target_user)
    
            similarity_df = calculate_similarity_for_recommendations(recommendations, tags_movies, df_feat, user_features,
                                                                     df_wsd, user_wsd)
    
            weights = {
                'wsd_similarity': 0.05,
                'wsd_confidence': 0.05,
                'pos_match': 0.3,
                'ner_match': 0.3,
                'sentiment_match': 0.2,
                'cluster_match': 0.1
            }
    
            # Add a check for empty DataFrame
            if similarity_df.empty:
                print(f"No recommendations for user {userId}, adding empty recommendation.")
                recommended_tags = []
            else:
                # Assuming similarity_df is your DataFrame
                similarity_df['total_similarity'] = similarity_df.apply(
                    lambda row: calculate_total_similarity(row, weights), axis=1)
    
                # Sort the DataFrame by 'total_similarity' in descending order
                sorted_similarity_df = similarity_df.sort_values(by='total_similarity', ascending=False)
    
                # Remove rows where 'movie_tag' is also present in 'user_tag' column
                filtered_similarity_df = sorted_similarity_df[~sorted_similarity_df['movie_tag'].isin(user_tags)]
    
                # Drop duplicates based on the 'movie_tag' column to get distinct tags
                distinct_tags_df = filtered_similarity_df.drop_duplicates(subset=['movie_tag'])
    
                # Get the top 10 distinct tags along with their movieId
                top_10_distinct_tags_with_movieId = distinct_tags_df[
                    ['movieId', 'movie_tag', 'ner_match', 'pos_match', 'cluster_match', 'total_similarity']].head(10)
                # Calculate the average similarity of the top 10 tags
                average_similarity = top_10_distinct_tags_with_movieId[
                    'total_similarity'].mean() if not top_10_distinct_tags_with_movieId.empty else 0
    
                recommended_tags = top_10_distinct_tags_with_movieId['movie_tag'].tolist()
    
            # Now, we can add the results to df_fold_1_results DataFrame, regardless of whether recommendations are empty or not.
            temp_df = pd.DataFrame({
                'userId': [userId],  # Make sure to pass a list even if it's a single userId
                'recommended_tags': [recommended_tags],
                # The brackets ensure it's treated as a single list entry in the DataFrame
                'average_similarity': [average_similarity]  # Add the average similarity for the top 10 tags
            })
    
            # Concatenate the temporary DataFrame with the main results DataFrame
            df_fold_1_results = pd.concat([df_fold_1_results, temp_df], ignore_index=True)
    
        except Exception as e:
            print(f"An error occurred for user {userId}: {e}")
    
    ### RESULTs
    df_fold_1_results.to_json(f'../dataset/evaluation/df_fold_{fold}_results.json')
    print(f'Finished Recommendation fold {fold}')


## Evaluate

In [ ]:
import pandas as pd
from collections import defaultdict
import numpy as np
from collections import Counter

# Function to calculate Precision@K, Recall@K, and NDCG@K
def calculate_metrics_at_k(test_data_path, results_data_path, k):
    # Load the data
    test_data = pd.read_csv(test_data_path)
    df_results = pd.read_json(results_data_path)

    # Process the test data into a mapping of userId to a list of actual tags
    test_tags_dict = defaultdict(set)
    for _, row in test_data.iterrows():
        test_tags_dict[row['userId']].add(row['tag'])

    # Initialize lists to store individual metrics
    precision_list = []
    recall_list = []
    ndcg_list = []

    for _, row in df_results.iterrows():
        user_id = row['userId']
        # Take the top K recommended tags
        recommended_tags = set(row['recommended_tags'][:k])  # Top K tags
        actual_tags = test_tags_dict.get(user_id, set())
        k_len = min(len(actual_tags),k)


        # # Precision@K
        individual_precisions = []
        for tag in recommended_tags:
            if tag in actual_tags:
                individual_precisions.append(1)  # Relevant tag in the top K recommendations
            else:
                individual_precisions.append(0)  # Non-relevant tag in the top K recommendations

        precision_at_k = sum(individual_precisions) / k_len  # Average precision
        precision_list.append(precision_at_k)

        # Recall@K
        relevant_tags = set(recommended_tags).intersection(actual_tags)
        if len(actual_tags) > 0:
            recall_at_k = len(relevant_tags) / len(actual_tags)
        else:
            recall_at_k = 1
        recall_list.append(recall_at_k)

    # Calculate overall metrics
    overall_precision_at_k = sum(precision_list) / len(precision_list) if precision_list else 0
    overall_recall_at_k = sum(recall_list) / len(recall_list) if recall_list else 0

    return overall_precision_at_k, overall_recall_at_k


# Function to calculate Coverage@K
def calculate_coverage_at_k(test_data_path, results_data_path, k):
    all_possible_tags = set()
    test_df = pd.read_csv(test_data_path)
    all_possible_tags.update(test_df['tag'].unique())

    df_results = pd.read_json(results_data_path)
    recommended_tags_set = set()

    for _, row in df_results.iterrows():
        recommended_tags = row['recommended_tags'][:k]
        recommended_tags_set.update(recommended_tags)

    coverage_at_k = len(recommended_tags_set) / len(all_possible_tags) if all_possible_tags else 0
    # print("all unique tags",len(all_possible_tags))
    return coverage_at_k


# Function to calculate Long-Tail Coverage@K
def calculate_LTcoverage_at_k(test_data_path, results_data_path, k):
    df_results = pd.read_json(results_data_path)
    df_test = pd.read_csv(test_data_path)

    # Get all tags from the test data and their frequencies
    test_tags = df_test['tag']
    tag_counts_test = Counter(test_tags)
    df_freq_test = pd.DataFrame(tag_counts_test.items(), columns=['Tag', 'Frequency'])

    if df_freq_test.empty:
        return 0.0  # Avoid division by zero if no test tags

    # Identify long-tail tags based on the frequency quantile in the test data
    threshold = df_freq_test['Frequency'].quantile(0.8)
    long_tail_tags = set(df_freq_test[df_freq_test['Frequency'] <= threshold]['Tag'])

    if not long_tail_tags:
        return 1.0 # If no long-tail tags, consider coverage as 100%

    # Collect all recommended tags up to k for all users
    recommended_tags_at_k = set()
    for _, row in df_results.iterrows():
        recommended_tags = row['recommended_tags'][:k]
        recommended_tags_at_k.update(recommended_tags)

    # Calculate the intersection of recommended tags and long-tail tags
    covered_long_tail_tags = recommended_tags_at_k.intersection(long_tail_tags)

    # Calculate long-tail coverage
    long_tail_coverage = len(covered_long_tail_tags) / len(long_tail_tags) if long_tail_tags else 0.0
    return long_tail_coverage


In [ ]:
metrics_data = []
# Set K
k_values = range(1,11)

# Loop through fold_0 to fold_4
for fold_num in range(5):
    test_data_path = f'../dataset/evaluation_folds/fold_{fold_num}_test.csv'
    results_data_path = f'../dataset/evaluation/testNMF_train_{fold_num}_recs.json'

    pre_k = []
    rec_k = []
    cov_k = []
    ltcov_k = []

    # Loop through each value of k and calculate precision and recall
    for k in k_values:
        # Calculate precision, recall, and NDCG for this fold and value of K
        pre, rec  = calculate_metrics_at_k(test_data_path, results_data_path, k)
        cov = calculate_coverage_at_k(test_data_path,results_data_path, k)
        ltcov = calculate_LTcoverage_at_k(test_data_path,results_data_path, k)

        # Append the results for this value of k
        pre_k.append(pre)
        rec_k.append(rec)
        cov_k.append(cov)
        ltcov_k.append(ltcov)
        # ndcg_k.append(ndcg)

    # Append results to the metrics data list
    metrics_data.append({
        'Fold': fold_num,
        **{f'P@{k}': pre_k[i] for i, k in enumerate(k_values)},  # Add Precision values for each k
        **{f'R@{k}': rec_k[i] for i, k in enumerate(k_values)},  # Add Recall values for each k
        **{f'F1@{k}': (2*rec_k[i]*pre_k[i])/(rec_k[i]+pre_k[i]) for i, k in enumerate(k_values)},  # Add Recall values for each k
        **{f'Cov@{k}': cov_k[i] for i, k in enumerate(k_values)},  # Add Recall values for each k
        **{f'LTCov@{k}': ltcov_k[i] for i, k in enumerate(k_values)},  # Add Recall values for each k
    })

# Create a DataFrame from the collected metrics data
df_metrics = pd.DataFrame(metrics_data)

# Add a row for the overall average metrics across all folds
avg_row = {
    'Fold': 'Avg.',
    **{f'P@{k}': df_metrics[f'P@{k}'].mean() for i, k in enumerate(k_values)},
    **{f'R@{k}': df_metrics[f'R@{k}'].mean() for i, k in enumerate(k_values)},
    **{f'F1@{k}': df_metrics[f'F1@{k}'].mean() for i, k in enumerate(k_values)},
    **{f'Cov@{k}': df_metrics[f'Cov@{k}'].mean() for i, k in enumerate(k_values)},
    **{f'LTCov@{k}': df_metrics[f'LTCov@{k}'].mean() for i, k in enumerate(k_values)},
}

std_row = {
    'Fold': 'Std.',
    **{f'P@{k}': df_metrics[f'P@{k}'].std() for i, k in enumerate(k_values)},
    **{f'R@{k}': df_metrics[f'R@{k}'].std() for i, k in enumerate(k_values)},
    **{f'F1@{k}': df_metrics[f'F1@{k}'].std() for i, k in enumerate(k_values)},
    **{f'Cov@{k}': df_metrics[f'Cov@{k}'].std() for i, k in enumerate(k_values)},
    **{f'LTCov@{k}': df_metrics[f'LTCov@{k}'].std() for i, k in enumerate(k_values)},
}

df_metrics = pd.concat([df_metrics, pd.DataFrame([avg_row])], ignore_index=True)
df_metrics = pd.concat([df_metrics, pd.DataFrame([std_row])], ignore_index=True)

# Output the table
df_metrics